In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset = "/content/drive/MyDrive/Leather Defect Classification"

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = dataset  # path to parent folder

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load all images and labels from the 6 subfolders
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes found:", dataset.classes)   # ['folding_marks', 'grain_off', ..., 'pinhole']
print("Number of images:", len(dataset))

# DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)


Classes found: ['Folding marks', 'Grain off', 'Growth marks', 'loose grains', 'non defective', 'pinhole']
Number of images: 3600


In [4]:
from torch.utils.data import random_split

# ----- 2. Train/Validation split -----
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [5]:
from torchvision import models

# ----- 3. Load pretrained model -----
model = models.efficientnet_b0(pretrained=True)

# ----- Freeze all layers -----
for param in model.parameters():
    param.requires_grad = False

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 118MB/s] 


In [6]:
import torch.nn as nn

# ----- 4. Replace classifier -----
num_classes = 6
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)

# Only classifier will be trained
params_to_update = model.classifier.parameters()

In [12]:
from torchsummary import summary

# ----- 5. Model summary -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
summary(model, (3, 224, 224))  # like model.summary() in Keras

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [7]:
import torch

# ----- 6. Setup device, loss, optimizer -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params_to_update, lr=1e-3)

In [14]:
# ----- 7. Training loop -----
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(outputs.argmax(1) == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(outputs.argmax(1) == labels.data)

    val_loss = val_loss / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1/10
Train Loss: 0.4211, Train Acc: 0.8319
Val Loss: 0.2797, Val Acc: 0.8931
Epoch 2/10
Train Loss: 0.4114, Train Acc: 0.8354
Val Loss: 0.2883, Val Acc: 0.8736
Epoch 3/10
Train Loss: 0.3834, Train Acc: 0.8472
Val Loss: 0.2798, Val Acc: 0.8944
Epoch 4/10
Train Loss: 0.3559, Train Acc: 0.8545
Val Loss: 0.2564, Val Acc: 0.9069
Epoch 5/10
Train Loss: 0.3626, Train Acc: 0.8538
Val Loss: 0.2756, Val Acc: 0.9014
Epoch 6/10
Train Loss: 0.3773, Train Acc: 0.8521
Val Loss: 0.2340, Val Acc: 0.9056
Epoch 7/10
Train Loss: 0.3432, Train Acc: 0.8608
Val Loss: 0.2277, Val Acc: 0.9167
Epoch 8/10
Train Loss: 0.3471, Train Acc: 0.8597
Val Loss: 0.2440, Val Acc: 0.8958
Epoch 9/10
Train Loss: 0.3379, Train Acc: 0.8684
Val Loss: 0.2330, Val Acc: 0.9125
Epoch 10/10
Train Loss: 0.3651, Train Acc: 0.8476
Val Loss: 0.2316, Val Acc: 0.9208


In [22]:
# torch.save(model, "leather_defect_model.pth")
torch.save(model.state_dict(), "leather_defect_model_state.pth")
